In [0]:
dbutils.widgets.text('filename','')
p_file_name = dbutils.widgets.get('filename')
display(p_file_name)

In [0]:
df = spark.read.format('parquet').load(f'abfss://source@databricksetestg.dfs.core.windows.net/{p_file_name}')
display(df)
df.count()

In [0]:
df_autoloader = spark.readStream.format('cloudFiles')\
    .option('cloudFiles.format','parquet')\
    .option('cloudFiles.schemaLocation',f'abfss://bronze@databricksetestg.dfs.core.windows.net/checkpoint_{p_file_name}')\
    .load(f'abfss://source@databricksetestg.dfs.core.windows.net/{p_file_name}')

In [0]:
df_autoloader.writeStream.format('delta')\
    .outputMode('append')\
    .option('checkpointLocation',f'abfss://bronze@databricksetestg.dfs.core.windows.net/checkpoint_{p_file_name}')\
    .trigger(once=True)\
    .option('path',f'abfss://bronze@databricksetestg.dfs.core.windows.net/{p_file_name}')\
    .start()